# `20180721a` - Introducing classes - version un0_e

In [1]:
#!/usr/bin/python
import spidev
import RPi.GPIO as GPIO
import time
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import json
import time
from pyUn0 import *

pyexiv2 does not exist on RPi


In [2]:
x = us_spi()
x.JSON = {}
x.JSON["firmware_md5"]="fa6a7560ade6d6b1149b6e78e0de051f"
x.JSON["firmware_version"]="e_un0"
x.JSON["data"]=[]
x.JSON["registers"]={}
x.JSON["parameters"]={}

x.StartUp()
x.ConfigSPI()

f = 0x00
x.WriteFPGA(0xED,f) # Frequency of ADC acquisition / sEEADC_freq (3 = 16Msps, 1 = 32, 0 = 64, 2 = 21Msps)

pyUn0.py:97: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(CS_FLASH,GPIO.OUT)


Reset GPIO 23 - Low 1s
Reset GPIO 23 - High 0.2s
spi.cshigh is False
spi mode is 1
spi maxspeed is 2000000hz
Pulse width: 200  ns --  0x19
Pulses delay: 100  ns --  0xc
Poff: 2000  ns --  0x1 0x0
Delay between: 7000 ns --  0x3 0x80
Config FPGA done!


In [3]:
#LoopSPI(spi)

In [3]:
x.TestSPI(3) # LED2 clignote 3x

In [4]:
x.JSON

{'data': [],
 'firmware_md5': 'fa6a7560ade6d6b1149b6e78e0de051f',
 'firmware_version': 'e_un0',
 'parameters': {'DeltaAcq': 7000,
  'DeltaAcq_Real': 114,
  'Poff': 2000,
  'Poff_Real': 32,
  'Pon': 200,
  'Pon_Real': 3,
  'PulsesDelay': 100,
  'PulsesDelay_Real': 1},
 'registers': {208: 12,
  224: 25,
  225: 1,
  226: 0,
  227: 3,
  228: 128,
  235: 0,
  236: 51,
  237: 0,
  238: 160}}

In [5]:
Curve = x.CreateDACCurve(0,1000,True)[0] # Beginning, Ending, Linear (if False, expo)
print Curve,len(Curve)
#setDACConstant(0x77,spi)
x.setDACCurve(Curve)

[0, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400, 425, 450, 475, 500, 525, 550, 575, 600, 625, 650, 675, 700, 725, 750, 775, 800, 825, 850, 875, 900, 925, 950, 0, 0] 41


In [8]:
N=0

In [9]:
x.JSON["experiment"]={}
x.JSON["experiment"]["id"] = "20180721a"
x.JSON["experiment"]["description"]="Classical experiment with calibration piezo"
x.JSON["experiment"]["target"] = "calibration rig"
x.JSON["experiment"]["position"] = "0"
x.JSON["V"]="25"

#x.WriteFPGA(0xEC,0x01) # Doing 10 lines
x.WriteFPGA(0xEB,0x00) # Doing one line if 0, several if 1
x.WriteFPGA(0xEC,0x01) # Doing 1 lines
if x.JSON["registers"][235]: # means it's set to 1, ie that's multiples lines
    NLines = x.JSON["registers"][236]
else:
    NLines = 1

In [10]:
# Experiment 
N = N+1
x.JSON["N"] = N

# Timings
t1 = 200
t2 = 100
t3 = 2000
t4 = 300-t1-t2+10
t5 = 200000

# DAC
Curve = x.CreateDACCurve(0,0,True)[0]
x.setDACCurve(Curve)
x.setPulseTrain(t1,t2,t3,t4,t5)
    
# Trigger
x.WriteFPGA(0xEF,0x01) # Cleaning memory pointer
x.WriteFPGA(0xEA,0x01) # Software Trig : As to be clear by software

Fech = int(64/((1+f)))
LAcq = t5/1000 #ns to us 
Nacq = LAcq * Fech * NLines

A = []
for i in range(2*Nacq+1):
    A.append ( x.spi.xfer([0x00] )[0] )
a = np.asarray(A).astype(np.int16)
x.JSON["data"] = A

with open(x.JSON["experiment"]["id"]+'-'+str(N)+".json", 'w') as outfile:
    json.dump(x.JSON, outfile)
    
print "Nacq=",Nacq,", LAcq=",LAcq,", Fech=",Fech,", NLines=",NLines 
print "Acquisition #"+str(N)+" done."

Pulse width: 200  ns --  0x19
Pulses delay: 295  ns --  0x25
Poff: 2484  ns --  0x1 0x3d
Delay between: 2970 ns --  0x1 0x7c
Acquisition length:  3373.824  us --  0x66 0xf6
Nacq= 12800 , LAcq= 200 , Fech= 64 , NLines= 1
Acquisition #1 done.
